In [36]:
import torch
import numpy as np
from sklearn.datasets import load_files
from glob import glob
import cv2 # type: ignore
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os

In [37]:
#狗的数据集
def load_data(path): 
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = torch.nn.functional.one_hot(torch.tensor(np.array(data['target'])).long(),133)  # 将类别转换成数组。
    print(dog_targets.shape)
    return dog_files,dog_targets
train_files,train_targets = load_data('../dogImages/train')
valid_files,valid_targets = load_data('../dogImages/valid')  
test_files,test_targets = load_data('../dogImages/test')
dog_names = [item[20:-1] for item in sorted(glob("../dogImages/train/*/"))]


torch.Size([6680, 133])
torch.Size([835, 133])
torch.Size([836, 133])


In [38]:
#人脸的数据集
random.seed(8675309)
human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)  #type:ignore

In [39]:
#人脸检测模型加载
face_cascade = cv2.CascadeClassifier('../haarcascades/haarcascade_frontalface_alt.xml') # type: ignore


In [40]:
def face_detector(img_path):
    img = cv2.imread(img_path) #type:ignore
    if img is not None:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #type:ignore
        faces = face_cascade.detectMultiScale(gray)
    
    #圈出模型识别出的人脸在图中位置。
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1) # type: ignore
    cv_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # type: ignore
    plt.imshow(cv_rgb)
    plt.show()
    plt.close()
    return faces
    

In [41]:
face_detection = []
for i in human_files:
    for j in os.listdir(i):
        face_detection.append(face_detector(os.path.join(i,j)))

In [42]:
human_files_short = human_files[:100]
dog_files_short = train_files[:100]

In [43]:
def model_ignores(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #type:ignore
    detect_face = len(face_cascade.detectMultiScale(gray)) # type: ignore
    return detect_face>0

In [44]:
error_num=0
for i in human_files_short:
    for j in os.listdir(i):
        detect_face = model_ignores(os.path.join(i,j))
        if(detect_face==False):
            error_num+=1;
            print("human_files[{}] detect human face fail".format(os.path.join(i,j)))
print("error check number:{}".format(error_num))
error_num=0
for i in dog_files_short:
    detect_face = model_ignores(i)
    if(detect_face==True):
        error_num+=1
        print("dog_files[{}] is detectted as human face".format(i))
print("error check number:{}".format(error_num))

error check number:0
dog_files[../dogImages/train\095.Kuvasz\Kuvasz_06442.jpg] is detectted as human face
dog_files[../dogImages/train\099.Lhasa_apso\Lhasa_apso_06646.jpg] is detectted as human face
dog_files[../dogImages/train\009.American_water_spaniel\American_water_spaniel_00628.jpg] is detectted as human face
dog_files[../dogImages/train\057.Dalmatian\Dalmatian_04023.jpg] is detectted as human face
dog_files[../dogImages/train\096.Labrador_retriever\Labrador_retriever_06474.jpg] is detectted as human face
dog_files[../dogImages/train\106.Newfoundland\Newfoundland_06989.jpg] is detectted as human face
dog_files[../dogImages/train\117.Pekingese\Pekingese_07559.jpg] is detectted as human face
dog_files[../dogImages/train\039.Bull_terrier\Bull_terrier_02805.jpg] is detectted as human face
dog_files[../dogImages/train\097.Lakeland_terrier\Lakeland_terrier_06516.jpg] is detectted as human face
dog_files[../dogImages/train\057.Dalmatian\Dalmatian_04086.jpg] is detectted as human face
dog

In [45]:
#使用ResNet-50模型
import torch.nn as nn

In [46]:
import torch
import torch.nn as nn
#第一层神经网络结构
class InitialBlock(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(InitialBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x
#残差神经网络结构
class BottleneckResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BottleneckResidualBlock, self).__init__()
        bottleneck_channels = out_channels // 4
        
        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)
        self.relu2 = nn.ReLU(inplace=True)

        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu3 = nn.ReLU(inplace=True)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        out += self.shortcut(residual)
        # 如果shortcut非空，应用shortcut并且调整输入的通道数以匹配输出
        if self.shortcut:
            residual = self.shortcut(residual)
        out += residual
        out = self.relu3(out)

        return out


In [47]:
#ResNet网络结构第一层和残差层继续组合
class ResidualStage(nn.Module):
    def __init__(self, in_channels, out_channels, block, blocks_num, stride=1):
        super(ResidualStage, self).__init__()
        self.blocks = nn.ModuleList([block(in_channels, out_channels, stride)] +
                                    [block(out_channels, out_channels) for _ in range(1, blocks_num)])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self,block,layers,num_classes=133):
        super(ResNet,self).__init__()
        self.initial_block=InitialBlock(in_channels=3, out_channels=64)
        self.stage0 = ResidualStage(in_channels=64, out_channels=256, block=block, blocks_num=layers[0])
        self.stage1 = ResidualStage(in_channels=256, out_channels=612, block=block, blocks_num=layers[1], stride=2)
        self.stage2 = ResidualStage(in_channels=612, out_channels=1024, block=block, blocks_num=layers[2], stride=2)
        self.stage3 = ResidualStage(in_channels=1024, out_channels=2048, block=block, blocks_num=layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)
    def forward(self,x):
        x = self.initial_block(x)
        x = self.stage0(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
    
# 创建 ResNet-50 模型
resnet50 = ResNet(BottleneckResidualBlock, [3, 4, 6, 3])

In [48]:
resnet50

ResNet(
  (initial_block): InitialBlock(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (stage0): ResidualStage(
    (blocks): ModuleList(
      (0): BottleneckResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, aff

In [55]:
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
# 定义预处理操作
transform = transform.Compose([
    transform.Resize((224, 224)),  # 调整图像大小
    transform.ToTensor(),          # 转换为张量格式
    transform.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 归一化
])
dataset = ImageFolder(root='../dogImages/train', transform=transform)  # type: ignore # 替换为你的数据集路径
# 创建数据加载器
dataloader = DataLoader(dataset, batch_size=32, shuffle=True,drop_last=True)

In [67]:
# for i in train_files:
s = cv2.imread(train_files[0])
print(s.shape)
print("`````````````````````````````````````````")
for i in dataloader:
    print(i[0].shape)
    break

(427, 640, 3)
`````````````````````````````````````````
torch.Size([32, 3, 224, 224])


In [56]:
for i in dataloader:
    print(i)

[tensor([[[[-0.0745, -0.1059, -0.0902,  ...,  0.4275,  0.3961,  0.0275],
          [-0.0902, -0.1765, -0.1922,  ...,  0.0196,  0.2627,  0.2235],
          [-0.1529, -0.2157, -0.2235,  ...,  0.2392,  0.1059,  0.1059],
          ...,
          [ 0.5922,  0.3490,  0.3176,  ..., -0.5373,  0.1529,  0.0510],
          [ 0.4667,  0.4510,  0.3569,  ..., -0.2627,  0.0039,  0.2784],
          [ 0.4196,  0.3804,  0.1216,  ..., -0.2627, -0.0275,  0.1373]],

         [[ 0.1765,  0.1843,  0.2078,  ...,  0.4118,  0.4039,  0.0667],
          [ 0.1922,  0.1294,  0.1216,  ..., -0.0667,  0.1922,  0.1765],
          [ 0.1216,  0.0667,  0.0745,  ...,  0.1765,  0.0510,  0.0431],
          ...,
          [ 0.6078,  0.4275,  0.4745,  ..., -0.4824,  0.2784,  0.2157],
          [ 0.5059,  0.5059,  0.4353,  ..., -0.2392,  0.0745,  0.3961],
          [ 0.4745,  0.4196,  0.1529,  ..., -0.2627, -0.0118,  0.2235]],

         [[-0.2863, -0.3333, -0.4510,  ...,  0.1529,  0.0588, -0.3412],
          [-0.2863, -0.4118, 

OSError: image file is truncated (148 bytes not processed)

In [15]:
#数据加载，前面加载了
train_targets[0].shape

torch.Size([133])

In [22]:
#定义损失函数，优化器和超参数
batch_size = 32
learning_rate=0.001
num_epochs=20
import torch.optim as optim
dloss=nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(),lr=learning_rate)


In [23]:
from collections.abc import Iterable
dataloader is Iterable

False

In [26]:
import matplotlib.pyplot as plt

In [33]:
train_losses = []
for epoch in range(num_epochs):
    resnet50.train()
    running_loss=0.0
    total_samples = 0
    print(epoch)
    try:
        for inputs,labels in dataloader:
                #梯度置零
                optimizer.zero_grad()
                #前向传播
                outputs=resnet50(inputs)
                #求损失
                loss = dloss(outputs, labels) # type: ignore
                #反向传播
                # 反向传播和优化
                loss.backward()
                optimizer.step()
                
                # 计算当前批次的损失并累加到 running_loss 中
                running_loss += loss.item() * inputs.size(0)
                
                # 统计总样本数量
                total_samples += inputs.size(0)
    except OSError as e:
        print(f"OSError occurred: {e}")
    # 计算当前epoch的平均损失
    epoch_loss = running_loss / total_samples
    train_losses.append(epoch_loss)
    
    # 输出当前epoch的损失
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    
# 绘制损失函数的图表
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

0


OSError: image file is truncated (148 bytes not processed)